In [1]:
# Import necessary libraries
from langchain_community.chat_models import ChatOllama
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
import pandas as pd
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure Ollama
llm = ChatOllama(
    model="qwen2.5-coder:7b",
    temperature=0, 
    
)


C:\Users\s7seg\AppData\Local\Temp\ipykernel_43324\1102744423.py:14: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [43]:

# Load and preprocess data - wrapped in try-except for better error visibility
try:
    # Load and preprocess data
    data = pd.read_csv("../data/app_data.csv")
    
    # Let's add some debug prints to see the data
    print("Initial data shape:", data.shape)
    print("\nFirst few rows of raw data:")
    display(data.head())  # Using display() which works better in Jupyter
    
    # Convert amount and balance columns, handling potential NaN values
    data['amount'] = pd.to_numeric(data['amount'].str.replace("QAR" , "").str.replace("," , ""), errors='coerce')
    data['balance'] = pd.to_numeric(data['balance'].str.replace("QAR" , "").str.replace("," , ""), errors='coerce')
    
    # Print after numeric conversion
    print("\nAfter numeric conversion:")
    display(data[['amount', 'balance']].head())
    
    # Convert date and time, handling potential errors
    data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y', errors='coerce')
    data['time'] = pd.to_datetime(data['time'], format='%H:%M', errors='coerce').dt.time
    
    # Print after date conversion
    print("\nAfter date conversion:")
    display(data[['date', 'time']].head())
    
except Exception as e:
    print(f"Error loading or preprocessing data: {str(e)}")
    raise



Initial data shape: (55, 12)

First few rows of raw data:


,message,amount,balance,date,store,time,Code,Label,Reason,processed_at,ner_duration,cls_duration
0,"Debit transaction of QAR 1,398.64 from Card **...","QAR 1,398.64","QAR 5,801.13",11/4/2024,QATAR CHARITY MPGS,13:43,4,Donations and Gifts,The transaction is related to donations and ch...,2/27/2025 13:23,NaN,NaN
1,Debit transaction of QAR 381.82 from Card ****...,QAR 381.82,"QAR 7,088.77",11/5/2024,Google *YouTube,2:25,2,Groceries,"The transaction is for groceries, which falls ...",2/27/2025 13:23,NaN,NaN
2,Debit transaction of QAR 49.84 from Card ****3...,QAR 49.84,"QAR 8,537.86",11/6/2024,HAJER FOOD TRADING DOHDOHA,20:11,2,Groceries,The store name 'Hajer Food Stuffs' is a grocer...,2/27/2025 13:23,NaN,NaN
3,Debit transaction of QAR 43.25 from Card ****1...,QAR 43.25,"QAR 4,041.09",11/8/2024,WOQOD - AL KHARAITIYATPSD,9:12,2,Groceries,The transaction is related to purchasing groce...,2/27/2025 13:23,NaN,NaN
4,Debit transaction of QAR 93.54 from Card ****2...,QAR 93.54,"QAR 8,058.37",11/10/2024,SHEZAN SUPERMARKET,21:31,2,Groceries,"The transaction is related to groceries, speci...",2/27/2025 13:23,NaN,NaN



After numeric conversion:


,amount,balance
0,1398.64,5801.13
1,381.82,7088.77
2,49.84,8537.86
3,43.25,4041.09
4,93.54,8058.37



After date conversion:


,date,time
0,2024-11-04,13:43:00
1,2024-11-05,02:25:00
2,2024-11-06,20:11:00
3,2024-11-08,09:12:00
4,2024-11-10,21:31:00


In [47]:
# Filter the dataframe for transactions in January 2024
df = data 
# Filter the dataframe for transactions in January 2024
january_expenses = df[df['date'].dt.to_period('M') == '2024-01']

# Find the highest expense
highest_expense = january_expenses['amount'].max()

print(f"The highest expense in January 2024 was QAR {highest_expense:.2f}")



The highest expense in January 2024 was QAR nan


In [49]:
df['date'].dt.to_period('M')

0     2024-11
1     2024-11
2     2024-11
3     2024-11
4     2024-11
5     2024-11
6     2024-11
7     2024-11
8     2024-11
9     2024-11
10    2024-11
11    2024-11
12    2024-11
13    2024-11
14    2024-11
15    2024-11
16    2024-11
17    2024-11
18    2024-12
19    2024-12
20    2024-12
21    2024-12
22    2024-12
23    2024-12
24    2024-12
25    2024-12
26    2024-12
27    2024-12
28    2024-12
29    2024-12
30    2024-12
31    2024-12
32    2024-12
33    2024-12
34    2024-12
35    2024-12
36    2024-12
37    2024-12
38    2024-12
39    2024-12
40    2024-12
41    2024-12
42    2024-12
43    2024-12
44    2024-12
45    2024-12
46    2024-12
47    2024-12
48    2024-12
49    2024-12
50    2024-12
51    2024-12
52    2024-12
53    2024-12
54    2024-11
Name: date, dtype: period[M]

In [20]:
df

,sms,card_number,amount,store,account,balance,date,time,month
0,"Debit transaction of QAR 1,298.13 from Card **...",****4953,1298.13,SMAISMA CAFE DOHQA,***704063,830.52,NaT,13:43:00,NaN
1,Debit transaction of QAR 413.48 from Card ****...,****9448,413.48,Qatar Charity doha,***663320,4188.75,NaT,02:25:00,NaN
2,Debit transaction of QAR 723.33 from Card ****...,****9858,723.33,MONOPRIX HYPERMARKET DOHDOHA,***459249,5390.72,NaT,20:11:00,NaN
3,Debit transaction of QAR 493.88 from Card ****...,****8611,493.88,GOOGLE *Forest Focus f,***682551,3835.47,NaT,09:12:00,NaN
4,Debit transaction of QAR 884.19 from Card ****...,****5724,884.19,AL MAHA AUTO SPARE PARDOHA,***303376,4988.11,NaT,21:31:00,NaN
...,...,...,...,...,...,...,...,...,...
359,"Debit transaction of QAR 1,429.01 from Card **...",****2219,1429.01,MINISTER OF FOREIGN AFDOHA,***511450,4335.05,NaT,05:11:00,NaN
360,Debit transaction of QAR 138.83 from Card ****...,****1223,138.83,SANDWHICH POORI KARA D,***418630,8602.99,NaT,06:00:00,NaN
361,Debit transaction of QAR 138.97 from Card ****...,****8807,138.97,ORANGE MINI HYPERMARKEDOHA,***696471,9385.77,NaT,10:14:00,NaN
362,Debit transaction of QAR 61.75 from Card ****9...,****9099,61.75,WOQOD - UMM EBAIRIYA SDRD,***329114,4186.89,NaT,NaT,NaN


In [31]:
# Create Pandas DataFrame Agent
agent = create_pandas_dataframe_agent(
    llm,
    data,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    return_intermediate_steps=False,  # Don't return intermediate steps
    max_iterations=3,  # Limit iterations to prevent infinite loops
)



# You can now run test queries like:
# test_query("What is the total number of rows in the dataset?")

In [32]:
import numpy as np 
from io import StringIO
from contextlib import redirect_stdout


def test_query(query):
    print(f"Testing query: {query}")
    try:
        response = agent.run(query)
        # If response contains code, try to execute it
        if isinstance(response, str) and "```python" in response:
            print(f"Response contains code: {response}")
            # Extract code between backticks
            code = response.split("```python")[1].split("```")[0].strip()
            # Create a local namespace with access to the data
            local_dict = {'df': data, 'pd': pd, 'np': np}
            # Execute the code, collect the output and return it 
            f = StringIO()
            with redirect_stdout(f):
                exec(code, globals(), local_dict)
            cod_res = f.getvalue()
            print(f"Code result: {cod_res}")
            return response, cod_res
        else:
            print(f"Response: {response}")
        return response
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None


In [27]:
res = test_query("what is the month with the highest total spending in 2024 ?") 
print(res)

Testing query: what is the month with the highest total spending in 2024 ?


> Entering new AgentExecutor chain...
Thought: To find the month with the highest total spending in 2024, I need to:
1. Filter the dataframe for transactions that occurred in 2024.
2. Group the filtered data by month.
3. Sum the amounts for each month.
4. Identify the month with the highest sum.

Action: Filter the dataframe for transactions that occurred in 2024.
Action Input: df[df['date'].dt.year == 2024]Filter the dataframe for transactions that occurred in 2024. is not a valid tool, try one of [python_repl_ast].It seems there was an error in the action input. Let's correct it and proceed with filtering the dataframe for transactions that occurred in 2024.

Action: Filter the dataframe for transactions that occurred in 2024.
Action Input: df[df['date'].dt.year == 2024]Filter the dataframe for transactions that occurred in 2024. is not a valid tool, try one of [python_repl_ast].It seems there was an error i

In [67]:
res[0]

"To calculate the average amount spent per month, you can follow these steps:\n\n1. Convert the `datetime` column to a pandas datetime object.\n2. Extract the year and month from the datetime object.\n3. Group the data by year and month.\n4. Calculate the mean of the `amount` column for each group.\n\nHere's how you can do it in Python using pandas:\n\n```python\nimport pandas as pd\n\n# Assuming df is your DataFrame\n\n# Convert the 'datetime' column to a datetime object\ndf['datetime'] = pd.to_datetime(df['datetime'])\n\n# Extract year and month from the datetime object\ndf['year_month'] = df['datetime'].dt.to_period('M')\n\n# Group by year and month and calculate the mean of the 'amount' column\navg_spent_per_month = df.groupby('year_month')['amount'].mean()\n\nprint(avg_spent_per_month)\n```\n\nThis will give you a Series where the index is the year-month period and the values are the average amounts spent in each month."

In [57]:
res = test_query("waht was the highes payment in 07/2024 ? mention the store, date, amount, and remaining balance ?")  # correct answer is  QR IBE QPAY doha , 2024-07-05, 3290.0 , 3,212.99 
print(res)

Testing query: waht was the highes payment in 07/2024 ? mention the store, date, amount, and remaining balance ?


> Entering new AgentExecutor chain...
To find the highest payment in July 2024 along with the store, date, amount, and remaining balance, you can use the following code:

```python
# Filter transactions for July 2024
july_2024_transactions = df[(df['date'].dt.month == 7) & (df['date'].dt.year == 2024)]

# Find the transaction with the highest amount
highest_payment = july_2024_transactions.loc[july_2024_transactions['amount'].idxmax()]

# Extract the required information
store = highest_payment['store']
date = highest_payment['date']
amount = highest_payment['amount']
balance = highest_payment['balance']

print(f"Store: {store}")
print(f"Date: {date}")
print(f"Amount: QAR {amount:.2f}")
print(f"Remaining Balance: QAR {balance:.2f}")
```

This code will filter the transactions for July 2024, find the transaction with the highest amount, and then extract and print the store 

In [58]:
res = test_query("How much was spent on SNOONU TRADING across all months?")# correct answer is  2182.00 
print(res)


Testing query: How much was spent on SNOONU TRADING across all months?


> Entering new AgentExecutor chain...
To calculate the total amount spent on SNOONU TRADING across all months, you can use the following code:

```python
# Filter the dataframe for transactions related to SNOONU TRADING
snoonu_transactions = df[df['store'] == 'SNOONU TRADING']

# Calculate the sum of the 'amount' column for these transactions
total_spent_on_snoonu = snoonu_transactions['amount'].sum()

print(f"Total amount spent on SNOONU TRADING: QAR {total_spent_on_snoonu:.2f}")
```

This code first filters the dataframe to include only rows where the 'store' column is equal to "SNOONU TRADING". Then, it calculates the sum of the 'amount' column for these filtered transactions. The result will give you the total amount spent on SNOONU TRADING across all months.

> Finished chain.
Response contains code: To calculate the total amount spent on SNOONU TRADING across all months, you can use the following code:

```p

In [59]:
res = test_query("what is the lowest spent amount ?")  
print(res)


Testing query: what is the lowest spent amount ?


> Entering new AgentExecutor chain...
To find the lowest spent amount in the `amount` column of your pandas DataFrame, you can use the `min()` function. Here's how you can do it:

```python
lowest_spent_amount = df['amount'].min()
print(lowest_spent_amount)
```

This will give you the smallest value in the `amount` column.

> Finished chain.
Response contains code: To find the lowest spent amount in the `amount` column of your pandas DataFrame, you can use the `min()` function. Here's how you can do it:

```python
lowest_spent_amount = df['amount'].min()
print(lowest_spent_amount)
```

This will give you the smallest value in the `amount` column.
Code result: 1.0

("To find the lowest spent amount in the `amount` column of your pandas DataFrame, you can use the `min()` function. Here's how you can do it:\n\n```python\nlowest_spent_amount = df['amount'].min()\nprint(lowest_spent_amount)\n```\n\nThis will give you the smallest value in t

In [60]:
res = test_query("what is the avg spent per month ?")  # correct answer is  QR IBE QPAY doha , 2024-07-05, 3290.0 , 3,212.99 
print(res)


Testing query: what is the avg spent per month ?


> Entering new AgentExecutor chain...
To calculate the average amount spent per month, you can follow these steps:

1. Convert the `datetime` column to a pandas datetime object.
2. Extract the year and month from the datetime object.
3. Group the data by year and month.
4. Calculate the mean of the `amount` column for each group.

Here's how you can do it in Python using pandas:

```python
import pandas as pd

# Assuming df is your DataFrame

# Convert the 'datetime' column to a datetime object
df['datetime'] = pd.to_datetime(df['datetime'])

# Extract year and month from the datetime object
df['year_month'] = df['datetime'].dt.to_period('M')

# Group by year and month and calculate the mean of the 'amount' column
avg_spent_per_month = df.groupby('year_month')['amount'].mean()

print(avg_spent_per_month)
```

This will give you a Series where the index is the year-month period and the values are the average amounts spent in each month.

In [64]:
from duckduckgo_search import DDGS

def search_store(store_name):
    """
    Search for store information using DuckDuckGo API
    Returns top 5 search results
    """
    if not store_name:
        return []

    try:
        # Use DuckDuckGo search
        results = DDGS().text(store_name, max_results=5)
        
        # Format results
        formatted_results = []
        for item in results:
            formatted_results.append({
                "name": item.get("title", ""),
                "snippet": item.get("body", ""),
            })
            
        return formatted_results

    except Exception as e:
        print(f"Search error: {str(e)}")
        return []

In [65]:
search_store("snooun trading")

[{'name': "Snoonu | Qatar's All-in-One Delivery App - Food, Electronics & More",
  'snippet': "Qatar's first delivery app for all your needs: food, groceries, gifts & personal errands. Order anything, get it delivered within minutes anywhere in Qatar."},
 {'name': 'SNOONU TRADING AND SERVICES - Dun & Bradstreet',
  'snippet': 'snoonu trading and services. d&b business directory home / business directory / information / telecommunications / wired and wireless telecommunications (except satellite) / qatar / lusail / snoonu trading and services; snoonu trading and services. website. get a d&b hoovers free trial.'},
 {'name': 'Snoonu - Doha, Qatar - Contact Number, Email Address',
  'snippet': 'Snoonu is a Qatar-based personal concierge and delivery company that was launched in two phases. The first phase, which began in April 2019, focuses on offline services like personal concierging and door-to-door deliveries.'},
 {'name': 'Snoonu: Fastest Delivery - Become A Partner | Fastest Food Del

In [72]:
max(data[(data.date.dt.month == 1) & (data.date.dt.year == 2025)]['amount'])

368.0